# Additional Analysis: Binary Semi-Major Axis and Orientation

This notebook adds the semi-major axis (`log_a` and `a`) and orientation angle (`alpha`) of the source binary to your final table, and performs a sanity check to ensure the projected separation is not less than the semi-major axis. It also provides histograms for these quantities.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load your previously saved table (update path if needed)
table = pd.read_csv('data/popsycle_table_with_calcs.csv')

# If you have the original filtered_df_with_calcs, you can use that instead.
# Otherwise, merge in the columns from your original DataFrame as needed.
# For this example, we'll assume you have the relevant columns in 'df'
from astropy.io import fits
fits_file = 'data/all_fields_Mrun_EWS_w_comp_params.fits'
with fits.open(fits_file) as hdul:
    data = hdul[1].data
    data_array = np.array(data).byteswap().view(data.dtype.newbyteorder('='))
    df = pd.DataFrame(data_array)

# Select only rows that are in the final table (e.g., by matching obj_id_L, obj_id_S, etc.)
# Here, we'll assume the order matches, but you may want to merge on unique keys.
df_filtered = df.loc[table.index]  # crude, but works if order is preserved

# Add comp_S_log_a and comp_S_alpha to the table
table['binary_log_a_au'] = df_filtered['comp_S_log_a']
table['binary_a_au'] = 10 ** df_filtered['comp_S_log_a']
table['binary_alpha_deg'] = df_filtered['comp_S_alpha']

# --- Sanity check: a (semi-major axis) should not be greater than sep (projected separation) ---
mask = table['binary_a_au'] > table['binary_sep_au']
print(f'Number of cases where a > sep: {mask.sum()} / {len(table)}')

# --- Histograms ---
plt.figure(figsize=(6,4))
plt.hist(table['binary_log_a_au'].dropna(), bins=50, alpha=0.7)
plt.xlabel('log10(a) [AU]')
plt.ylabel('Count')
plt.title('Histogram of binary_log_a_au')
plt.show()

plt.figure(figsize=(6,4))
plt.hist(table['binary_sep_au'].dropna(), bins=50, alpha=0.7)
plt.xlabel('Projected Separation (AU)')
plt.ylabel('Count')
plt.title('Histogram of binary_sep_au')
plt.show()

# Save the updated table
table.to_csv('data/popsycle_table_with_calcs_and_a.csv', index=False)

## New Columns Added

| Your Column Name      | Original Column(s)   | Units                        |
| :---                 | :---                 | :---                         |
| `binary_log_a_au`    | `comp_S_log_a`       | log10(semi-major axis/AU)    |
| `binary_a_au`        | `comp_S_log_a`       | Semi-major axis (AU)         |
| `binary_alpha_deg`   | `comp_S_alpha`       | Orientation angle (deg)      |

- `binary_log_a_au`: log10 of the semi-major axis of the source binary (AU)
- `binary_a_au`: semi-major axis of the source binary (AU)
- `binary_alpha_deg`: orientation angle of the source binary (deg)

**Sanity check:** The semi-major axis (`a`) should not be greater than the projected separation (`sep`). The code above prints the number of cases where this is violated.

**Histograms:** The notebook also plots histograms for `binary_log_a_au` and `binary_sep_au` to help you visually inspect the distributions.